## In this notebook

- Get predictions based on means per station and tod, and create data for submission.

In [1]:
import os
import json
import shutil
import datetime

# analytics
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
import numpy as np

# model
from tensorflow import keras
from keras.models import load_model

# plot
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

/opt/venv/lib/python3.8/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/venv/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/opt/venv/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/venv/lib/python3.8/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/venv/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/venv/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: cannot open shared object file: No such file or directory']
  warnings.warn(f"file system plugin

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]

In [3]:
DATA_FOLDER = "../data"

TIMESTEP = 96
TEST_DATA_DAYS_CNT = 19
TEST_DATA_MIN_DATE = "2021-02-19 00:00:00"
TEST_DATA_MAX_DATE = "2021-03-10 23:45:00"

TARGETS = ["Available", "Charging", "Passive", "Other"]
SUBMISSIONS_FOLDER = "../data/submissions/"

In [4]:
! ls -l $DATA_FOLDER

total 184816
-rw-r--r-- 1 nobody nogroup    570468 Nov 28 10:44 20221128104433_submissions.zip
-rw-r--r-- 1 nobody nogroup    562307 Nov 28 14:03 20221128140318_submissions.zip
-rw-r--r-- 1 nobody nogroup    562307 Nov 28 14:16 20221128141653_submissions.zip
-rw-r--r-- 1 nobody nogroup    579606 Nov 29 21:26 20221129212605_submissions.zip
-rw-r--r-- 1 nobody nogroup    530021 Nov 29 21:45 20221129214542_submissions.zip
-rw-r--r-- 1 nobody nogroup    576681 Nov 30 20:46 20221130204613_submissions.zip
drwxr-xr-x 6 nobody nogroup       192 Nov 29 08:39 predictions
drwxr-xr-x 5 nobody nogroup       160 Nov 28 10:06 submissions
-rw-rw-r-- 1 nobody nogroup  14095077 Aug  4 08:41 test.csv
-rw-rw-r-- 1 nobody nogroup 171754453 Aug  4 08:41 train.csv


## Load data

In [5]:
# load train data

df = pd.read_csv(os.path.join(DATA_FOLDER, "train.csv"))
df.columns = [col.lower() for col in df.columns]
df = df.sort_values(by="date").reset_index(drop=True)

df.tail()

,date,station,available,charging,passive,other,tod,dow,trend,latitude,longitude,postcode,area
1851078,2021-02-18 23:45:00,FR*V75*EBELI*32*1,3,0,0,0,95,5,22202,48.84477,2.352420,75005,south
1851079,2021-02-18 23:45:00,FR*V75*EBELI*31*1,2,1,0,0,95,5,22202,48.85104,2.345493,75005,south
1851080,2021-02-18 23:45:00,FR*V75*EBELI*30*1,2,0,1,0,95,5,22202,48.84571,2.345032,75005,south
1851081,2021-02-18 23:45:00,FR*V75*EBELI*39*1,1,0,1,1,95,5,22202,48.85267,2.313680,75007,west
1851082,2021-02-18 23:45:00,FR*V75*EBELI*99*1,3,0,0,0,95,5,22202,48.87780,2.391820,75019,east


In [6]:
# train models

models = {}

df["station-tod"] = df.station + "-" + df.tod.astype(str)

for y_col in [tar.lower() for tar in TARGETS]:
    models[y_col] = (
        df
        .groupby(["station-tod"])[y_col]
        .mean()
        .to_dict()
    )

## Create predictions

In [7]:
df_test = pd.read_csv(os.path.join(DATA_FOLDER, "test.csv"))
df_test.tail()

,date,Station,tod,dow,trend,Latitude,Longitude,Postcode,area
165979,2021-03-10 23:45:00,FR*V75*EBELI*95*1,95,4,24122,48.86851,2.295469,75016,west
165980,2021-03-10 23:45:00,FR*V75*EBELI*96*1,95,4,24122,48.83550,2.349650,75013,south
165981,2021-03-10 23:45:00,FR*V75*EBELI*97*1,95,4,24122,48.83995,2.397822,75012,east
165982,2021-03-10 23:45:00,FR*V75*EBELI*98*1,95,4,24122,48.84998,2.361920,75004,south
165983,2021-03-10 23:45:00,FR*V75*EBELI*99*1,95,4,24122,48.87780,2.391820,75019,east


In [8]:
# predict

df_test["station-tod"] = df_test.Station + "-" + df.tod.astype(str)

for y_col in [tar.lower() for tar in TARGETS]:
    df_test[y_col] = (
        df_test["station-tod"]
        .map(models[y_col])
    )    

In [9]:
# join preds on test data

df_test = (
    df_test
    .rename(
        {
            "available": "Available", 
            "charging": "Charging", 
            "passive": "Passive", 
            "other": "Other",
        }, 
        axis=1
    )
    .drop(["station-tod"], axis=1)
)

df_test[TARGETS] = df_test[TARGETS].round()

df_test.tail()

,date,Station,tod,dow,trend,Latitude,Longitude,Postcode,area,Available,Charging,Passive,Other
165979,2021-03-10 23:45:00,FR*V75*EBELI*95*1,95,4,24122,48.86851,2.295469,75016,west,1.0,0.0,0.0,2.0
165980,2021-03-10 23:45:00,FR*V75*EBELI*96*1,95,4,24122,48.83550,2.349650,75013,south,2.0,0.0,0.0,0.0
165981,2021-03-10 23:45:00,FR*V75*EBELI*97*1,95,4,24122,48.83995,2.397822,75012,east,2.0,0.0,0.0,0.0
165982,2021-03-10 23:45:00,FR*V75*EBELI*98*1,95,4,24122,48.84998,2.361920,75004,south,3.0,0.0,0.0,0.0
165983,2021-03-10 23:45:00,FR*V75*EBELI*99*1,95,4,24122,48.87780,2.391820,75019,east,2.0,0.0,0.0,0.0


In [10]:
df_test.dtypes

date          object
Station       object
tod            int64
dow            int64
trend          int64
Latitude     float64
Longitude    float64
Postcode       int64
area          object
Available    float64
Charging     float64
Passive      float64
Other        float64
dtype: object

In [11]:
# convert data in the right format

df_test["date"] = pd.to_datetime(df_test["date"])
df_test["Postcode"] = df_test["Postcode"].astype(str)

## Process predictions

In [12]:
df_test["plugs_sum"] = (
    df_test
    .loc[:, TARGETS]
    .sum(axis=1)
)

df_test.plugs_sum.value_counts()

3.0    105070
2.0     58140
4.0      2774
Name: plugs_sum, dtype: int64

In [13]:
# add other column values

df_test["Available"] = (
    df_test
    .Available
    .mask(
        df_test.plugs_sum.eq(2),
        df_test.Available.add(1)
    )
)

In [14]:
df_test["plugs_sum"] = (
    df_test
    .loc[:, TARGETS]
    .sum(axis=1)
)

df_test.plugs_sum.value_counts()

3.0    163210
4.0      2774
Name: plugs_sum, dtype: int64

In [15]:
# get rid of plugs_sum == 4

# check distributions
(
    df_test
    .loc[df_test.plugs_sum == 4]
    .apply(
        lambda row: f"{int(row['Available'])}-{int(row['Charging'])}-{int(row['Passive'])}-{int(row['Other'])}",
        axis=1
    )
    .value_counts()
)

2-0-1-1    1197
2-1-0-1     836
2-1-1-0     741
dtype: int64

In [16]:
df_test["Other"] = (
    df_test
    .Other
    .mask(
        df_test.plugs_sum.eq(4),
        0
    )
)

In [17]:
df_test["plugs_sum"] = (
    df_test
    .loc[:, TARGETS]
    .sum(axis=1)
)

df_test.plugs_sum.value_counts()

3.0    165243
4.0       741
Name: plugs_sum, dtype: int64

In [18]:
# get rid of plugs_sum == 4

# check distributions
(
    df_test
    .loc[df_test.plugs_sum == 4]
    .apply(
        lambda row: f"{int(row['Available'])}-{int(row['Charging'])}-{int(row['Passive'])}-{int(row['Other'])}",
        axis=1
    )
    .value_counts()
)

2-1-1-0    741
dtype: int64

In [19]:
df_test["Passive"] = (
    df_test
    .Passive
    .mask(
        df_test.plugs_sum.eq(4),
        0
    )
)

In [20]:
df_test["plugs_sum"] = (
    df_test
    .loc[:, TARGETS]
    .sum(axis=1)
)

df_test.plugs_sum.value_counts()

3.0    165984
Name: plugs_sum, dtype: int64

## Create submission

In [21]:
# create CSV on station level

(
    df_test
    .loc[:, ["date", "area", "Station"] + TARGETS]
    .to_csv(os.path.join(SUBMISSIONS_FOLDER, "station.csv"), index=False)
)

In [22]:
# create CSV on area level

(
    df_test
    .groupby(["date", "area"])[TARGETS]
    .sum()
    .reset_index()
    .to_csv(os.path.join(SUBMISSIONS_FOLDER, "area.csv"), index=False)
)

In [23]:
# create CSV on area level

(
    df_test
    .groupby(["date"])[TARGETS]
    .sum()
    .reset_index()
    .to_csv(os.path.join(SUBMISSIONS_FOLDER, "global.csv"), index=False)
)

In [24]:
# add CSV files into ZIP archive 

dt_now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

shutil.make_archive(
    base_name=f"/usr/src/app/data/{dt_now}_submissions", 
    format="zip", 
    root_dir="/usr/src/app/data/submissions/"
)

'/usr/src/app/data/20221130205124_submissions.zip'

## Results

- The simplest model had the best results. :/